# Übung 8 - JobPosts Similarity Implementation of Topic Modeling LDA algorithm (Experiment + Improved Version)
In this notebook the LDA (Latent Dirichlet Allocation) algorithmen will be experimented on.
For more information see down.



-------------

#### 1. Import Libaries
For more Information see:  
- gensim: https://radimrehurek.com/gensim/
- pyLDAvis: https://pyldavis.readthedocs.io/en/latest/readme.html

In [2]:
!pip install --upgrade gensim #install gensim libary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
!pip install pyldavis #install pyLDAVIS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=c154cd89d72a378f634a8b93cbefa9cecee888acbb76d499fadac689e83d38b3
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyldavis


In [4]:
!pip install openpyxl #install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora
from nltk.corpus import stopwords
import spacy
import pyLDAvis
from gensim.models import LdaModel
import pyLDAvis.gensim_models as gensimvis
from enum import Enum

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


-------------

#### 2. Load Data Set

In [6]:
df = pd.read_excel("./data_scientist_united_states_job_postings_jobspikr.xlsx")
df

,crawl_timestamp,url,job_title,category,company_name,city,state,country,inferred_city,inferred_state,...,job_description,job_type,salary_offered,job_board,geo,cursor,contact_email,contact_phone_number,uniq_id,html_job_description
0,2019-02-06 06:26:22,https://www.indeed.com/viewjob?jk=fd83355c2b23...,Enterprise Data Scientist I,Accounting/Finance,Farmers Insurance Group,Woodland Hills,CA,Usa,Woodland hills,California,...,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549432819114777,NaN,NaN,3b6c6acfcba6135a31c83bd7ea493b18,NaN
1,2019-02-06 06:33:41,https://www.dice.com/jobs/detail/Data-Scientis...,Data Scientist,NaN,Luxoft USA Inc,Middletown,NJ,Usa,Middletown,New jersey,...,We have an immediate opening for a Sharp Data ...,Undefined,NaN,dice,usa,1549432819122106,NaN,NaN,741727428839ae7ada852eebef29b0fe,NaN
2,2019-02-06 06:33:35,https://www.dice.com/jobs/detail/Data-Scientis...,Data Scientist,NaN,Cincinnati Bell Technology Solutions,New York,NY,Usa,New york,New york,...,Candidates should have the following backgroun...,Full Time,NaN,dice,usa,1549432819236156,NaN,NaN,cdc9ef9a1de327ccdc19cc0d07dbbb37,NaN
3,2019-02-06 06:33:42,https://www.indeed.com/viewjob?jk=841edd86ead2...,"Data Scientist, Aladdin Wealth Tech, Associate...",Accounting/Finance,BlackRock,New York,NY 10055 (Midtown area),Usa,New york,New york,...,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549432819259473,NaN,NaN,1c8541cd2c2c924f9391c7d3f526f64e,NaN
4,2019-02-06 06:48:23,https://job-openings.monster.com/senior-data-s...,Senior Data Scientist,biotech,CyberCoders,Charlotte,NC,Usa,Charlotte,North carolina,...,We are seeking an extraordinary Data Scientist...,Full Time,NaN,monster,usa,1549436429015957,NaN,NaN,445652a560a5441060857853cf267470,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2019-09-29 02:55:15,https://stackoverflow.com/jobs/271829,Data Scientist,Data Scientist,Upstart,Columbus,NaN,US,Columbus,Wisconsin,...,Join Upstart and help build the future of cred...,Full Time,$100k - 135k | Equity,Stackoverflow,NaN,1569916870107016,NaN,NaN,e9f2ca58c7abcd4553a4a0be9602312e,<h2>About this job</h2>_x000D_\n<p><span>Compe...
9996,2019-10-01 02:50:33,https://stackoverflow.com/jobs/283895,"Data Scientist, (Staff or Principal)",Data Scientist,Realtor.com,Santa Clara,NaN,US,Santa clara,New mexico,...,"Data Scientist, (Staff or Principal) at realto...",Full Time,NaN,Stackoverflow,NaN,1569916870211396,NaN,NaN,efe05f2364ee3df943e952bc68ed95a7,<h2>About this job</h2>_x000D_\n<p><span>Locat...
9997,2019-09-26 02:53:10,https://stackoverflow.com/jobs/293326,"Siri - Embedded Data Scientist, Data Organization","Data Scientist, Embedded Developer",Apple Inc.,Pittsburgh,NaN,US,Pittsburgh,Pennsylvania,...,Job Summary: Would you like to play a part in ...,Full Time,NaN,Stackoverflow,NaN,1569916870956579,NaN,NaN,b980054dbb6d3fe1c6a421a237d1495f,NaN
9998,2019-09-27 02:41:31,https://stackoverflow.com/jobs/283895,"Data Scientist, (Staff or Principal)",Data Scientist,Realtor.com,Santa Clara,NaN,US,Santa clara,New mexico,...,"Data Scientist, (Staff or Principal) at realto...",Full Time,NaN,Stackoverflow,NaN,1569916871226964,NaN,NaN,4b86da9d2b24b87ec3c815d9072ad616,<h2>About this job</h2>_x000D_\n<p><span>Locat...


#### 2.1 Load different numbers of samples and remove Nan-values


In [ ]:
c = df["job_description"].iloc[0:1000] #Change here the sample size (1000) for better testing
corpus = list(c.dropna())
#corpus

Im diesem Ansatz wird nur das Feature "job_description" experimentiert, da man versuchen möchte ganze Beschreibungen Themen zuzuordnen. Da man im letzten Notebook um Features behandelt wurden, die keien Sätze besitzen, versucht man mit dem LDA Ansatz, die ähnliche Texte in selbe Themen zuzuordnen.

#### 2.2 Preprocess the input documents

In [ ]:
#A small function to preproceses documents
def clean_text(documents):
  nlp = spacy.load("en_core_web_sm")
  new_corpus = []
  for document in documents:
    filler = ""
    doc = nlp(document)
    for token in doc:
      if token.is_stop:
        continue 
        
      if token.is_alpha:
        filler += " " +  token.lemma_

    new_corpus += [filler[1:len(filler)]]

  return new_corpus

In [ ]:
new_corpus = clean_text(corpus)

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use sele

In [ ]:
#new_corpus

#### 2.3 Tokenize documents
For more Information see:
- gensim simple_preprocess(): https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html

In [24]:
#A small function to tokenize the text
def preprocess(documents):
  new_documents = []
  for document in documents:
    tokens = gensim.utils.simple_preprocess(document) #convert text into token like spacy
    new_documents += [tokens]

  return new_documents


In [25]:
new_corpus2 = preprocess(new_corpus)

In [ ]:
#new_corpus2[2][0:20]

#### 2.4 Transform words into vectors (Bag-of-words approach)
Assign unique id's to each word.
For more information see:
- gensim corpora.Dictionary(): https://radimrehurek.com/gensim/corpora/dictionary.html
- transform into vectors: https://radimrehurek.com/gensim/auto_examples/core/run_corpora_and_vector_spaces.html

In [41]:
dictionary = corpora.Dictionary(new_corpus2)
def transform(documents):
  corpus = []
  for document in documents:
    vector = dictionary.doc2bow(document) #transform each text to vectors
    corpus += [vector]

  return corpus

In [42]:
new_corpus3 = transform(new_corpus2)

In [ ]:
#dictionary.token2id

In [53]:
#new_corpus3[0][0:20] #returns (id of the word , frequency of the word)

-------------------------

#### 3. Initalize LDA Model and train it
For more information see:
- gensim LDA model: https://radimrehurek.com/gensim/models/ldamodel.html
- training model: https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html

In [47]:
model = LdaModel(
    corpus=new_corpus3,
    id2word=dictionary,
    alpha='auto',
    eta='auto',
    iterations=400,
    num_topics=10,
    eval_every=None)

#### 4. Visualize results
for more information see:
- pyLDAvis prepare(): https://pyldavis.readthedocs.io/en/latest/modules/API.html

In [ ]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, new_corpus3, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.164330 -0.003804       1        1  58.365802
6      0.164289  0.015169       2        1  16.638998
8      0.163138 -0.042304       3        1  13.273737
2      0.138049 -0.016073       4        1   6.509414
7      0.118166  0.067140       5        1   5.038018
4     -0.053715 -0.027142       6        1   0.087135
1     -0.114423 -0.014314       7        1   0.035092
0     -0.179212  0.009155       8        1   0.034967
9     -0.222216 -0.000832       9        1   0.009609
5     -0.178405  0.013005      10        1   0.007228, topic_info=            Term          Freq         Total Category  logprob  loglift
16           and  21679.000000  21679.000000  Default  30.0000  30.0000
192           to  13987.000000  13987.000000  Default  29.0000  29.0000
145         pron  11183.000000  11183.000000  Default  28.0000  28.0000
121           of   8943.000000   8943.000000  Default  27.0000  27.0000
189          the  10448.000000  10448.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
126           or      0.017999   4154.363682  Topic10  -7.4303  -2.8145
109      machine      0.016370   1739.364380  Topic10  -7.5252  -2.0387
376  information      0.015365    813.796092  Topic10  -7.5885  -1.3425
27         build      0.014994   1054.826281  Topic10  -7.6130  -1.6264
26            be      0.015649   7527.523015  Topic10  -7.5702  -3.5487

[770 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
6362      1  0.238111  absorb
6362      5  0.714334  absorb
1266      1  0.631464  access
1266      2  0.164730  access
1266      3  0.076874  access
...     ...       ...     ...
224       5  0.033663    year
224       6  0.000748    year
224       7  0.000748    year
5679      3  0.828554    yolo
5680      3  0.779803   yourâ

[1496 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 9, 3, 8, 5, 2, 1, 10, 6])

**Ergebnis**
Wie man in der Visualisierung sehen kann, hat der LDA Algorithmus eigentlich nur 5 Themen ausgewählt, da die restlichen fünf Themen (7-10) Ausreißer darstelllen. Was man auch in der Visualiserierung sehen kann, ist, das die zugeordneten Wörter zu den jeweiligen Themen, eher Stoppwörter sind bzw. am meisten zufällige Wörter, die dem Kontext der Beschreibung keinen Sinn ergeben anzutreffen. Wenn man spezifische Wörter rausfiltert und vielleich spezifisches Preprocessing verwendet kann man den Topic Modeling LDA Ansatz verwenden um (hier für komplexere Dokumente, die Sätze haben) Themen zuzuweisen, die man mit herkömlichen Ähnlichkeitsmaße nicht machen könnte.

----------------------

## Experiment 1: Verbesserung des LDA Algorithmus. Das Preprocessing verbessern.

In [7]:
from gensim.models import Phrases
from gensim.models import phrases
from gensim.models.phrases import Phraser
from gensim.models import TfidfModel

#### Adding bigrams and trigrams to remove common words (Using Phraser from genism libary)
For more information see:
- helpful source: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#9createbigramandtrigrammodels
- Phrase (collocation) detection: https://radimrehurek.com/gensim/models/phrases.html

Bigram Implementation:

In [101]:
#iniate bigram model
bigram = Phrases(new_corpus2, 
min_count=5, #gnore all words and bigrams with total collected count lower than this value.
threshold = 30, #epresent a score threshold for forming the phrases (higher means fewer phrases)
connector_words=phrases.ENGLISH_CONNECTOR_WORDS)  #This will cause phrases to include common English articles, prepositions and conjuctions, such as bank_of_america or eye_of_the_beholder.


bigram_mod = Phraser(bigram) #Decorate model with Phraser to make it faster

Trigram Implementation:

In [102]:
#iniate trigram model
trigram = Phrases(bigram[new_corpus2], #call Phrase (here bigram) on dataset with Phrase[corpus]
min_count=5, 
threshold= 30,
connector_words=phrases.ENGLISH_CONNECTOR_WORDS)  

trigram_mod = gensim.models.phrases.Phraser(trigram)

In [103]:
# A small function to pass sentences to bigram and trigram models
def use_NGram(texts):
  new_texts = []
  for sentence in texts:
    new_texts += trigram_mod[bigram_mod[sentence]] #call model by: model[data]

  return new_texts


In [104]:
transformed_corpus = use_NGram(new_corpus2)
#transformed_corpus

#### Use TF - IDF model
For more information see:
- gensim tfidfmodel: https://radimrehurek.com/gensim/models/tfidfmodel.html

In [108]:
transformed_corpus2 = preprocess(transformed_corpus)
dictionary2 = corpora.Dictionary(transformed_corpus2)

def Use_Tfidf(documents):
  
  corpus = []
  new_corpus = []
  for document in documents:
    vector = dictionary2.doc2bow(document) #transform each text to vectors
    corpus += [vector]

  model = TfidfModel(corpus)

  #for document in corpus:
  new_corpus += model[corpus]



  return new_corpus

In [109]:
new_transformed_corpus = Use_Tfidf(transformed_corpus2)

#### Initalize LDA Model and train it

In [110]:
model2 = LdaModel(
    corpus=new_transformed_corpus,
    id2word=dictionary2,
    alpha='auto',
    eta='auto',
    iterations=400,
    num_topics=10,
    eval_every=None)

#### Visualize results

In [111]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model2, new_transformed_corpus, dictionary2)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.440342 -0.052811       1        1  13.170081
5      0.004071  0.440388       2        1  12.744294
2     -0.041735 -0.051790       3        1  11.215088
7     -0.055677 -0.049027       4        1   9.607429
9     -0.054031 -0.050111       5        1   9.597906
4     -0.057716 -0.048359       6        1   9.416216
8     -0.058355 -0.047505       7        1   9.349026
6     -0.058510 -0.047746       8        1   8.609615
1     -0.058608 -0.047277       9        1   8.516388
3     -0.059782 -0.045762      10        1   7.773956, topic_info=           Term         Freq        Total Category  logprob  loglift
65        datum  7664.000000  7664.000000  Default  30.0000  30.0000
126  experience  6112.000000  6112.000000  Default  29.0000  29.0000
32         data  5929.000000  5929.000000  Default  28.0000  28.0000
1          work  4815.000000  4815.000000  Default  27.0000  27.0000
4          team  2979.000000  2979.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
314      review   204.520384   205.883099  Topic10  -4.3798   2.5478
46     function   191.224531   192.587247  Topic10  -4.4470   2.5473
335      status   170.889329   172.252044  Topic10  -4.5594   2.5464
701       query   170.674016   172.036732  Topic10  -4.5607   2.5464
317     physics   153.624544   154.987260  Topic10  -4.6659   2.5456

[330 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
195       6  0.999416        ability
140       1  0.998132           able
122       5  0.992515       academic
388       4  0.989865        achieve
115       9  0.992926         action
...     ...       ...            ...
635       5  0.998036  visualization
229       6  0.992900           well
1         4  0.999651           work
26       10  0.998050          world
127       8  0.999069           year

[300 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 3, 8, 10, 5, 9, 7, 2, 4])

**Conclusion:** Wie man oben in der Visualisierung sehen kann, hat sich die Themenaufteilung und Themenzuteilung der Wörter in den Sätzen, deutlich verbessert. Es gibt fast keine Wörter, die keine Bedeutung bieten im ersten Ansatz sondern es gibt deutlich mehr Wörter, die Sinn zu dem Kontext der JobPosts bieten.

--------------------------------------

# Class for better experimenting 

In [8]:
#class for easy experimenting
class Topic():

  def __init__(self, n=10, iterations=400, eval_every=None, doc=[], preprocess=False, weight=False):
    self.n = n #number of topic default = 10
    self.iterations= iterations
    self.eval_every = eval_every
    self.doc = doc #actual documents
    self.p = preprocess 
    self.w = weight


    self.dictionary = None #save dictionary for LDA model
    self.model = None #save LDA model

    self.bi_object = None #save bigram object
    self.bi = None #save Bigram

    self.tri_object = None #save bigram object

  def __clean_text(self):
    nlp = spacy.load("en_core_web_sm")
    new_corpus = []
    for document in self.doc:
      filler = ""
      Doc = nlp(document)
      for token in Doc:
        if token.is_stop:
          continue 
          
        if token.is_alpha:
          filler += " " +  token.lemma_

      new_corpus += [filler[1:len(filler)]]

    self.doc = new_corpus


  def __use(self):
    new_documents = []
    for document in self.doc:
      tokens = gensim.utils.simple_preprocess(document) #convert text into token like spacy
      new_documents += [tokens]

    self.doc = new_documents

  def __use_NGram(self):
    new_texts = []
    for sentence in self.doc:
      new_texts += self.tri[self.bi[sentence]] #call model by: model[data]

    self.doc = new_texts


  def __bigramm(self):
    bigram = Phrases(self.doc, min_count=5, threshold = 30, connector_words=phrases.ENGLISH_CONNECTOR_WORDS)
    self.bi_object = bigram
    self.bi = Phraser(bigram) 

  def __trigramm(self):
    trigram = Phrases(self.bi_object[self.doc], min_count=5, threshold= 30, connector_words=phrases.ENGLISH_CONNECTOR_WORDS)  
    self.tri = gensim.models.phrases.Phraser(trigram)

  def __use_Tfidf(self):
    self.dictionary = corpora.Dictionary(self.doc)

    corpus = []
    new_corpus = []
    for document in self.doc:
      vector = self.dictionary.doc2bow(document) #transform each text to vectors
      corpus += [vector]

    model = TfidfModel(corpus)

    new_corpus += model[corpus]

    self.doc = new_corpus



  
  def __transform(self):
    self.dictionary = corpora.Dictionary(self.doc)

    corpus = []
    for document in self.doc:
      vector = self.dictionary.doc2bow(document) #transform each text to vectors
      corpus += [vector]

    self.doc = corpus

  
  def __instate_model(self):
    model = LdaModel(corpus=self.doc, id2word=self.dictionary, alpha='auto', eta='auto', iterations=self.iterations, num_topics=self.n, eval_every=self.eval_every)
    self.model = model

  
  def train(self):
    if self.p == True:
      self.__clean_text()
      self.__use()
      self.__transform()
      self.__instate_model()

    if self.w == True:
      self.__use()
      self.__bigramm()
      self.__trigramm()
      self.__use_NGram()
      self.__use()
      self.__use_Tfidf()
      self.__instate_model()

    else:
      self.__use()
      self.__transform()
      self.__instate_model()
      


  
  def visualize(self):
    if self.model != None:
      pyLDAvis.enable_notebook()
      vis = gensimvis.prepare(self.model, self.doc, self.dictionary)
      return vis

    else:
      print("Model not train yet!")


    

  

#### Experiment 2: Topic Modeling on Feature "category"
Um den Rahmen nicht zu sprengen, wird die Anzahl an Themen zwischen 1 bis 10 gewählt jedoch wird die Anzahl an Themen meistens auf 10 bleiben.

In [78]:
category = df["category"].iloc[0:1000] #Change here the sample size (1000) for better testing
corpus_category = list(category.dropna())
#corpus

In [87]:
topic_category = Topic(doc=corpus_category)
topic_category.train()
topic_category.visualize()

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.243971  0.045834       1        1  28.267216
3      0.168564 -0.036014       2        1  18.386936
4     -0.221697  0.381931       3        1  13.912682
6     -0.269386 -0.211994       4        1   9.778346
5     -0.185512 -0.141555       5        1   7.554250
7      0.127870 -0.096716       6        1   6.357331
9     -0.232852  0.071759       7        1   5.259883
8      0.272589  0.006024       8        1   3.656627
1      0.165044  0.095471       9        1   3.537560
2     -0.068591 -0.114741      10        1   3.289167, topic_info=             Term        Freq       Total Category  logprob  loglift
4        internet  148.000000  148.000000  Default  30.0000  30.0000
3        computer  147.000000  147.000000  Default  29.0000  29.0000
13   architecture  133.000000  133.000000  Default  28.0000  28.0000
14    engineering  133.000000  133.000000  Default  27.0000  27.0000
9             and  215.000000  215.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
17           life    0.083031   37.275486  Topic10  -6.5696  -2.6923
20         social    0.082970   37.937516  Topic10  -6.5703  -2.7107
18       physical    0.082945   37.274272  Topic10  -6.5706  -2.6933
33     technology    0.082913    6.721719  Topic10  -6.5710  -0.9808
6   entertainment    0.082890   19.306783  Topic10  -6.5713  -2.0362

[466 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
0         2  0.095435    accounting
0         4  0.222681    accounting
0         5  0.286305    accounting
0         6  0.254493    accounting
0         7  0.063623    accounting
...     ...       ...           ...
26        1  0.735730      training
26       10  0.183933      training
45        7  0.434697    translator
57        3  0.401138        travel
60        1  0.867661  volunteering

[180 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 5, 7, 6, 8, 10, 9, 2, 3])

In [ ]:
topic_category = Topic(doc=corpus_category)
topic_category.train()
topic_category.visualize()

**Conclusion:**

#### Experiment 3: Topic Modeling on Feature "state"
Ist es möglich mit Topic Modeling bzw. LDA Algorithmus auch Geografische Information in Themen einzuordnen?

In [90]:
state = df["state"].iloc[0:1000] #Change here the sample size (1000) for better testing
corpus_state = list(state.dropna())
topic_state = Topic(doc=corpus_state, preprocess=True)
topic_state.train()
topic_state.visualize()

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use sele

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.084508 -0.062328       1        1  16.420568
7     -0.241050 -0.034073       2        1  15.801483
6      0.143495 -0.010779       3        1  10.225276
9      0.155299 -0.053208       4        1   9.852486
1     -0.051576  0.142722       5        1   9.589451
2      0.057201  0.217320       6        1   9.512793
4      0.144194 -0.212402       7        1   8.121909
8      0.069307  0.121967       8        1   7.731869
3     -0.116284 -0.127449       9        1   6.898549
5     -0.076077  0.018231      10        1   5.845616, topic_info=      Term       Freq      Total Category  logprob  loglift
13      tx  68.000000  68.000000  Default  30.0000  30.0000
6       va  61.000000  61.000000  Default  29.0000  29.0000
0       nj  38.000000  38.000000  Default  28.0000  28.0000
9       ma  43.000000  43.000000  Default  27.0000  27.0000
11      wa  39.000000  39.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
29      tn   0.206080   8.789516  Topic10  -5.5119  -0.9136
25      ia   0.206016   5.091094  Topic10  -5.5123  -0.3678
18  market   0.206008  15.434714  Topic10  -5.5123  -1.4770
43      id   0.205993   5.460400  Topic10  -5.5124  -0.4380
14      oh   0.205980   5.554680  Topic10  -5.5124  -0.4551

[475 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
66       10  0.641988         afb
57        3  0.572402  alexandria
30        9  0.879872          ar
63        5  0.534289   arboretum
2         1  0.209905        area
...     ...       ...         ...
49        1  0.580621          wi
49        2  0.193540          wi
20        1  0.198580     wilmore
20        4  0.595739     wilmore
58        3  0.572401       wrest

[193 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 8, 7, 10, 2, 3, 5, 9, 4, 6])

**Conclusion:** Das LDA Modell scheint nicht geeignet zu sein, um geografische Lagen einzuteilen, da man in der Visualisierung sehen kann, dass die meisten Zuordnungen der Städte keinen Sinn ergeben oder die Namen sind einfach Buchstaben und bilden kein realen Orte dar. Das Preprocessing scheint keine Wirkung auf das Resulat zu haben.

#### Experiment 3: Topic Modeling on Feature "country"
Kann der LDA Algorithmus auch Länder in Themen einteilen?


In [95]:
country = df["country"]#Change here the sample size (1000) for better testing
corpus_country = list(country.dropna())
country.value_counts()

US               5384
Usa              4614
United States       2
Name: country, dtype: int64

In [96]:
topic_country = Topic(n=3, doc=corpus_country)
topic_country.train()
topic_country.visualize()

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0     -0.348358  0.0       1        1  48.834772
2      0.312720  0.0       2        1  39.984387
1      0.035638  0.0       3        1  11.180841, topic_info=     Term         Freq        Total Category  logprob  loglift
0     usa  4740.000000  4740.000000  Default   4.0000   4.0000
1      us  5204.000000  5204.000000  Default   3.0000   3.0000
2  states    28.000000    28.000000  Default   2.0000   2.0000
3  united    28.000000    28.000000  Default   1.0000   1.0000
1      us  4847.562151  5204.022846   Topic1  -0.0076   0.6458
2  states     0.248343    28.819425   Topic1  -9.8868  -4.0373
3  united     0.248343    28.819425   Topic1  -9.8868  -4.0373
0     usa    36.394557  4740.337970   Topic1  -4.8994  -4.1527
0     usa  3985.912316  4740.337970   Topic2  -0.0033   0.7433
2  states     0.496432    28.819425   Topic2  -8.9942  -3.1447
3  united     0.496432    28.819425   Topic2  -8.9942  -3.1447
1      us    12.333440  5204.022846   Topic2  -5.7815  -5.1282
2  states    28.074650    28.819425   Topic3  -3.6847   2.1648
3  united    28.074650    28.819425   Topic3  -3.6847   2.1648
0     usa   718.031097  4740.337970   Topic3  -0.4431   0.3036
1      us   344.127256  5204.022846   Topic3  -1.1786  -0.5252, token_table=      Topic      Freq    Term
term                         
2         3  0.971567  states
3         3  0.971567  united
1         1  0.931587      us
1         2  0.002306      us
1         3  0.066103      us
0         1  0.007594     usa
0         2  0.840868     usa
0         3  0.151466     usa, R=4, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

**Conclusion** Wie im letzen Beispiel, kann der LDA Algorithmus auch nicht mit Ländern umgehen. In diesem Fall gibt es zwei verschiedene Namen für USA aber diese aufgeteilt und allgemein die Länder nicht gut eingeteilt aber man darf dies nicht als Vorraussetzung sehen, da das Model eher drauf angelegt ist, einzelne Themen in Dokumentn aufzuspüren, als die Distanzen zu Ländern zu bestimmen.

#### Experiment 4: Topic Modeling on Feature "job_title"
Kann der LDA Algorithmus in diesem Beispielt mit job_title umgehen?

In [97]:
title = df["job_title"]#Change here the sample size (1000) for better testing
corpus_title = list(title.dropna())

In [135]:
topic_title = Topic(n=10, doc=corpus_title)
topic_title.train()
topic_title.visualize()

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.046962 -0.060695       1        1  22.013560
3     -0.072288  0.029815       2        1  15.644810
4     -0.036654 -0.082182       3        1  12.617945
6      0.076909 -0.157901       4        1   9.561588
7     -0.063499  0.039308       5        1   8.694541
0     -0.003908 -0.002734       6        1   7.452149
8     -0.102930  0.133154       7        1   6.911879
5     -0.017881  0.101512       8        1   6.906822
1     -0.029519 -0.081303       9        1   6.180064
9      0.296732  0.081025      10        1   4.016642, topic_info=          Term         Freq        Total Category  logprob  loglift
8       senior  1616.000000  1616.000000  Default  30.0000  30.0000
36          in  1011.000000  1011.000000  Default  29.0000  29.0000
22          sr   726.000000   726.000000  Default  28.0000  28.0000
54   principal   473.000000   473.000000  Default  27.0000  27.0000
60          ca   358.000000   358.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
12    learning    92.048771   462.416659  Topic10  -2.8118   1.6006
85        lead    38.097790   499.960231  Topic10  -3.6940   0.6404
0         data   160.740754  9671.685622  Topic10  -2.2543  -0.8824
2    scientist   158.171512  9412.181983  Topic10  -2.2704  -0.8714
100         ai     6.068140    97.171545  Topic10  -5.5311   0.4413

[453 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
259       8  0.960454  actuarial
546       4  0.971531         ad
115       1  0.034767   advanced
115       2  0.017383   advanced
115       5  0.956090   advanced
...     ...       ...        ...
789       8  0.851578         ww
343       6  0.757413      years
137       1  0.007875       york
137       2  0.015750       york
137       3  0.968612       york

[689 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 5, 7, 8, 1, 9, 6, 2, 10])

Kann dies mit Bigram, Trigram und Tf-idf- Verfahren verbessert werden?

In [134]:
topic_title2 = Topic(n=10, doc=corpus_title, weight=True)
topic_title2.train()
topic_title2.visualize()

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.272971  0.361466       1        1  28.905971
8      0.324943 -0.315320       2        1  27.969311
5      0.074716 -0.003192       3        1   8.827814
2     -0.095071 -0.007508       4        1   5.499446
3     -0.072488 -0.012855       5        1   5.458786
0      0.051233 -0.003895       6        1   5.087259
1     -0.121211 -0.004659       7        1   4.686030
6     -0.146212  0.016690       8        1   4.622768
9     -0.145675 -0.023897       9        1   4.547790
4     -0.143207 -0.006829      10        1   4.394825, topic_info=           Term         Freq        Total Category  logprob  loglift
2     scientist  9152.000000  9152.000000  Default  30.0000  30.0000
1          data  8932.000000  8932.000000  Default  29.0000  29.0000
8        senior  1639.000000  1639.000000  Default  28.0000  28.0000
36           in  1305.000000  1305.000000  Default  27.0000  27.0000
54    principal   855.000000   855.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
915  cerner_ccl     0.205842     1.598305  Topic10  -8.9403   1.0752
56         risk     0.223915    47.225422  Topic10  -8.8562  -2.2267
20           sr     0.232969   630.375152  Topic10  -8.8166  -4.7784
50       intern     0.214879    67.828503  Topic10  -8.8974  -2.6299
67      analyst     0.214878   145.910982  Topic10  -8.8974  -3.3959

[444 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
111       4  0.982444        advanced
91        6  0.429713         advisor
241       9  0.629833           agent
100       5  0.988966              ai
67        5  0.986903         analyst
...     ...       ...             ...
228       2  0.843113  vice_president
257       4  0.472327        virginia
17        1  0.932220              vp
149       5  0.724340           watch
28        6  0.960892            with

[198 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 9, 6, 3, 4, 1, 2, 7, 10, 5])

**Conclusion:** Obwohl man die Wörter mit einem Bigramm, Trigramm und TF-idf Model filtert, gibt es im zweiten Beispiel viel mehr Wörter, die keine Bedeutung haben oder keinenn Sinn ergeben. Tatsächlich ist im diesem Experiment, das erste Beispiel besser als der zweite direkt folgende Vefahren. Trotzdem, hat das Model, nicht so gut die Themen für die Wörter gefunden.

#### Experiment 5: Topic Modeling on Feature "job_type"

In [132]:
ttype = df["job_type"]#Change here the sample size (1000) for better testing
corpus_type = list(ttype.dropna())
ttype.value_counts()

Undefined     6109
Full Time     3395
Contract       488
Part Time        6
Internship       2
Name: job_type, dtype: int64

In [133]:
topic_type = Topic(n=5, doc=corpus_type)
topic_type.train()
topic_type.visualize()

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.262522 -0.040076       1        1  42.746249
4     -0.372411 -0.310032       2        1  39.636348
1     -0.292598  0.346959       3        1   6.743286
3      0.254014 -0.036821       4        1   5.983743
0      0.148472  0.039970       5        1   4.890374, topic_info=         Term         Freq        Total Category  logprob  loglift
0   undefined  5307.000000  5307.000000  Default   6.0000   6.0000
2        time  3559.000000  3559.000000  Default   5.0000   5.0000
1        full  3447.000000  3447.000000  Default   4.0000   4.0000
3    contract   967.000000   967.000000  Default   3.0000   3.0000
4        part   115.000000   115.000000  Default   2.0000   2.0000
5  internship     4.000000     4.000000  Default   1.0000   1.0000
1        full  2863.170154  3447.286891   Topic1  -0.6935   0.6642
2        time  2863.436307  3559.538946   Topic1  -0.6934   0.6323
5  internship     0.176524     4.071672   Topic1 -10.3875  -2.2885
4        part     0.175687   115.925884   Topic1 -10.3922  -5.6421
3    contract     0.377702   967.173494   Topic1  -9.6268  -6.9981
0   undefined     1.087959  5307.003009   Topic1  -8.5689  -7.6426
0   undefined  5284.386149  5307.003009   Topic2  -0.0051   0.9212
5  internship     0.679573     4.071672   Topic2  -8.9640  -0.8649
3    contract    23.199656   967.173494   Topic2  -5.4335  -2.8048
4        part     0.174888   115.925884   Topic2 -10.3213  -5.5711
1        full     1.595938  3447.286891   Topic2  -8.1102  -6.7525
2        time     1.631065  3559.538946   Topic2  -8.0884  -6.7627
3    contract   893.077468   967.173494   Topic3  -0.0118   2.6169
5  internship     0.711264     4.071672   Topic3  -7.1472   0.9519
4        part     0.708868   115.925884   Topic3  -7.1505  -2.4004
0   undefined     4.671241  5307.003009   Topic3  -5.2650  -4.3387
1        full     2.433228  3447.286891   Topic3  -5.9172  -4.5595
2        time     2.065755  3559.538946   Topic3  -6.0810  -4.7553
1        full   399.481244  3447.286891   Topic4  -0.6968   0.6609
2        time   399.191793  3559.538946   Topic4  -0.6975   0.6282
5  internship     0.223842     4.071672   Topic4  -8.1838  -0.0847
4        part     0.223082   115.925884   Topic4  -8.1872  -3.4370
3    contract     1.637078   967.173494   Topic4  -6.1940  -3.5653
0   undefined     1.124411  5307.003009   Topic4  -6.5697  -5.6434
4        part   114.643360   115.925884   Topic5  -1.7434   3.0068
5  internship     2.280469     4.071672   Topic5  -5.6608   2.4382
2        time   293.214026  3559.538946   Topic5  -0.8043   0.5214
1        full   180.606326  3447.286891   Topic5  -1.2889   0.0689
3    contract    48.881591   967.173494   Topic5  -2.5958   0.0329
0   undefined    15.733250  5307.003009   Topic5  -3.7294  -2.8031, token_table=      Topic      Freq        Term
term                             
3         2  0.023781    contract
3         3  0.923309    contract
3         4  0.002068    contract
3         5  0.050663    contract
1         1  0.830508        full
1         2  0.000580        full
1         3  0.000580        full
1         4  0.115743        full
1         5  0.052505        full
5         2  0.245599  internship
5         3  0.245599  internship
5         5  0.491199  internship
4         3  0.008626        part
4         5  0.992013        part
2         1  0.804318        time
2         2  0.000562        time
2         3  0.000562        time
2         4  0.112093        time
2         5  0.082314        time
0         1  0.000188   undefined
0         2  0.995666   undefined
0         3  0.000942   undefined
0         4  0.000188   undefined
0         5  0.003015   undefined, R=6, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 2, 4, 1])

Kann dies mit Bigram, Trigram und Tf-idf- Verfahren verbessert werden?

In [136]:
topic_type2 = Topic(n=5, doc=corpus_type, weight=True)
topic_type2.train()
topic_type2.visualize()

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.225639 -0.302192       1        1  38.876982
2     -0.245672  0.363047       2        1  24.482112
0      0.314080  0.044398       3        1  22.550859
4      0.256215 -0.009457       4        1   8.606507
1     -0.098984 -0.095797       5        1   5.483541, topic_info=         Term         Freq        Total Category  logprob  loglift
2        time  3302.000000  3302.000000  Default   6.0000   6.0000
0   undefined  5334.000000  5334.000000  Default   5.0000   5.0000
1        full  3194.000000  3194.000000  Default   4.0000   4.0000
3    contract  1034.000000  1034.000000  Default   3.0000   3.0000
4        part   498.000000   498.000000  Default   2.0000   2.0000
5  internship    37.000000    37.000000  Default   1.0000   1.0000
0   undefined  5197.049504  5334.155796   Topic1  -0.0025   0.9187
5  internship     0.198283    37.533348   Topic1 -10.1764  -4.2985
1        full    11.676547  3194.971036   Topic1  -6.1007  -4.6670
4        part     0.198227   498.010505   Topic1 -10.1767  -6.8842
3    contract     0.286899  1034.186121   Topic1  -9.8069  -7.2452
2        time     0.494531  3302.142853   Topic1  -9.2625  -7.8617
2        time  3260.014718  3302.142853   Topic2  -0.0064   1.3944
5  internship     0.165431    37.533348   Topic2  -9.8951  -4.0172
0   undefined    19.802458  5334.155796   Topic2  -5.1101  -4.1889
4        part     0.165380   498.010505   Topic2  -9.8954  -6.6029
3    contract     0.233587  1034.186121   Topic2  -9.5501  -6.9883
1        full     0.466314  3194.971036   Topic2  -8.8588  -7.4250
1        full  3020.279932  3194.971036   Topic3  -0.0006   1.4332
5  internship     0.155116    37.533348   Topic3  -9.8773  -3.9994
4        part     0.155053   498.010505   Topic3  -9.8777  -6.5852
3    contract     0.307177  1034.186121   Topic3  -9.1940  -6.6323
2        time     0.698322  3302.142853   Topic3  -8.3728  -6.9720
0   undefined     0.444803  5334.155796   Topic3  -8.8238  -7.9026
3    contract  1014.124678  1034.186121   Topic4  -0.1287   2.4331
1        full   119.380127  3194.971036   Topic4  -2.2681  -0.8344
5  internship     0.861335    37.533348   Topic4  -7.1997  -1.3218
0   undefined    15.865824  5334.155796   Topic4  -4.2863  -3.3651
4        part     0.861088   498.010505   Topic4  -7.2000  -3.9075
2        time     2.265039  3302.142853   Topic4  -6.2328  -4.8321
4        part   496.630756   498.010505   Topic5  -0.3918   2.9006
5  internship    36.153183    37.533348   Topic5  -3.0119   2.8660
0   undefined   100.993206  5334.155796   Topic5  -1.9846  -1.0634
3    contract    19.233780  1034.186121   Topic5  -3.6430  -1.0813
1        full    43.168115  3194.971036   Topic5  -2.8346  -1.4008
2        time    38.670243  3302.142853   Topic5  -2.9446  -1.5438, token_table=      Topic      Freq        Term
term                             
3         4  0.980481    contract
3         5  0.018372    contract
1         1  0.003756        full
1         3  0.945235        full
1         4  0.037246        full
1         5  0.013459        full
5         4  0.026643  internship
5         5  0.959147  internship
4         4  0.002008        part
4         5  0.997971        part
2         2  0.987238        time
2         3  0.000303        time
2         4  0.000606        time
2         5  0.011811        time
0         1  0.974287   undefined
0         2  0.003749   undefined
0         4  0.003000   undefined
0         5  0.018935   undefined, R=6, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 5, 2])

**Conclusion:** Der LDA Algorithmus kann an dem Feature "job_title" angewendet werden und wenn man den Bigramm, Trigramm und Tf-idf Verfahren zum granulieren verwendet, dann kann man wie im zweiten Beispiel dieses Experimentes sehen, das dies ein aktzeptaples Ergebnis liefert.

**End Result:** In allgemein, kann das Topic Modeling gut eingesetzt werden um ähnliche Job Posts zu finden, aber diese sollte dann feature bekommen, die ganze Sätze beinhalten und nicht nur einzelne Wörter. Man muss beachten, dasss das Model versucht die inhaltliche Ähnlichkeiten zu finden aber nicht die semantische Ähnlichkeit! Deswegen sollte man am besten das Topic Modeling (hier der LDA) an großen Mengen an Dokumenten mit viel Text statt mit einzelnen Kategorien.

-----------------------

#### Experiment 6: Wie müssen wir die Anzahl an Themen definieren, um ein aktzeptables Ergebnis zu bekommen?
In diesem Experiment fokussiert man sich nur auf ein Feauture: "Job_description". Man schaut wie gut die Themen zu den Wörtern passen und ebenfalls andersherum. Lässt sich Muster und Struckturen in den Themen finden? 
Rahmenbedingungen: Anzahl an Themen zwischen 5 und 30, da eine höhere Anzahl zu massigen Laufzeiten führen kann.

In [18]:
description = df["job_description"].iloc[0:1000] #only few samples
corpus_description = list(description.dropna())

Number of topics: 5

In [19]:
topic_description = Topic(n=5, doc=corpus_description)
topic_description.train()
topic_description.visualize()

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.059927  0.013023       1        1  61.782546
1     -0.040106 -0.001611       2        1  16.209251
0     -0.053671 -0.025440       3        1  15.156966
2     -0.026639  0.015361       4        1   6.741394
3      0.180343 -0.001333       5        1   0.109843, topic_info=        Term          Freq         Total Category  logprob  loglift
18       and  23249.000000  23249.000000  Default  30.0000  30.0000
91        in   8144.000000   8144.000000  Default  29.0000  29.0000
229     with   6563.000000   6563.000000  Default  28.0000  28.0000
49      data   9965.000000   9965.000000  Default  27.0000  27.0000
200       to  13081.000000  13081.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
129       or      0.864109   4535.332818   Topic5  -6.2948  -1.7518
111  machine      0.716244   1505.242794   Topic5  -6.4825  -0.8366
285   strong      0.662875    952.931495   Topic5  -6.5600  -0.4568
8        all      0.666506   1108.861881   Topic5  -6.5545  -0.6029
230     work      0.697420   2288.791418   Topic5  -6.5092  -1.2823

[438 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
236       1  0.658731  ability
236       2  0.112678  ability
236       3  0.139643  ability
236       4  0.089564  ability
3400      1  0.132515      aci
...     ...       ...      ...
235       1  0.768618     your
235       2  0.109574     your
235       3  0.062385     your
235       4  0.059186     your
235       5  0.000800     your

[793 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 1, 3, 4])

In [20]:
topic_description = Topic(n=5, doc=corpus_description, weight=True)
topic_description.train()
topic_description.visualize()

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.035022 -0.001650       1        1  36.272215
1      0.403866 -0.016033       2        1  18.445155
3     -0.167875 -0.325342       3        1  16.075969
0     -0.143896  0.329658       4        1  15.517461
4     -0.057072  0.013367       5        1  13.689200, topic_info=               Term          Freq         Total Category  logprob  loglift
16              and  20220.000000  20220.000000  Default  30.0000  30.0000
44             data   9729.000000   9729.000000  Default  29.0000  29.0000
14              the   9534.000000   9534.000000  Default  28.0000  28.0000
6                of   7978.000000   7978.000000  Default  27.0000  27.0000
39               in   7312.000000   7312.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
28    opportunities    411.017588    411.810618   Topic5  -4.6760   1.9866
185         project    408.075816    408.868925   Topic5  -4.6832   1.9866
841           based    408.011102    408.804217   Topic5  -4.6833   1.9866
987      statistics    402.149745    402.942774   Topic5  -4.6978   1.9866
1031       position    402.148430    402.941634   Topic5  -4.6978   1.9866

[180 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
387       4  0.999235     ability
338       4  0.997608      across
122       4  0.998942    advanced
469       1  0.998676  algorithms
60        2  0.998316         all
...     ...       ...         ...
234       2  0.999386     working
40        3  0.996523       world
221       1  0.999147       years
332       4  0.999840         you
18        3  0.999297        your

[150 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 4, 1, 5])

Number of topics: 10

In [24]:
topic_description = Topic(n=10, doc=corpus_description, weight=True)
topic_description.train()
topic_description.visualize()

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.456842  0.031210       1        1  29.444111
1     -0.015882 -0.440927       2        1  10.052589
5     -0.029329  0.084096       3        1   9.849560
4     -0.041134  0.056565       4        1   8.753134
2     -0.053151  0.049311       5        1   8.244380
3     -0.057964  0.046818       6        1   7.546022
8     -0.060585  0.045457       7        1   7.245898
7     -0.058266  0.046193       8        1   7.122086
6     -0.066174  0.043502       9        1   6.519776
0     -0.074359  0.037775      10        1   5.222443, topic_info=               Term          Freq         Total Category  logprob  loglift
44             data  12552.000000  12552.000000  Default  30.0000  30.0000
16              and  23578.000000  23578.000000  Default  29.0000  29.0000
14              the  11899.000000  11899.000000  Default  28.0000  28.0000
39               in   9541.000000   9541.000000  Default  27.0000  27.0000
184            with   7112.000000   7112.000000  Default  26.0000  26.0000
..              ...           ...           ...      ...      ...      ...
372  best_practices    179.067941    180.214935  Topic10  -4.5432   2.9458
134         testing    162.182817    163.329811  Topic10  -4.6423   2.9452
385             out    154.566936    155.713930  Topic10  -4.6904   2.9448
168           basic    131.551224    132.698218  Topic10  -4.8516   2.9435
151             phd    122.222824    123.369818  Topic10  -4.9251   2.9429

[330 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
387       9  0.999320   ability
172       4  0.994665      able
277      10  0.995490     about
149       4  0.990632  academic
484       3  0.993383   achieve
...     ...       ...       ...
40        8  0.996785     world
221       5  0.999362     years
332       3  0.999616       you
514       7  0.993208    you_ll
18        9  0.998872      your

[300 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 2, 6, 5, 3, 4, 9, 8, 7, 1])

Number of topics: 15

In [22]:
topic_description = Topic(n=15, doc=corpus_description, weight=True)
topic_description.train()
topic_description.visualize()

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.353441 -0.275903       1        1  10.699547
7      0.231950  0.382832       2        1   9.442379
13     0.010340  0.001213       3        1   7.583429
14     0.015130  0.002399       4        1   7.551934
10     0.012631  0.001796       5        1   7.540922
12     0.003958 -0.000315       6        1   7.481139
3     -0.025291 -0.006295       7        1   6.831130
8     -0.038139 -0.008554       8        1   6.273136
11    -0.049675 -0.010241       9        1   6.241312
4     -0.054416 -0.011026      10        1   6.177264
0     -0.071266 -0.013192      11        1   5.962747
9     -0.084517 -0.014650      12        1   5.116573
6     -0.097332 -0.015848      13        1   4.718864
5     -0.100155 -0.016018      14        1   4.391375
2     -0.106658 -0.016199      15        1   3.988252, topic_info=              Term          Freq         Total Category  logprob  loglift
16             and  27851.000000  27851.000000  Default  30.0000  30.0000
10              to  16002.000000  16002.000000  Default  29.0000  29.0000
44            data  14008.000000  14008.000000  Default  28.0000  28.0000
6               of  11452.000000  11452.000000  Default  27.0000  27.0000
14             the  12415.000000  12415.000000  Default  26.0000  26.0000
..             ...           ...           ...      ...      ...      ...
529          email    114.972811    116.356431  Topic15  -4.7167   3.2099
358  effectiveness     89.307644     90.691264  Topic15  -4.9693   3.2064
638         groups     74.368392     75.752011  Topic15  -5.1523   3.2034
225    corporation     52.638089     54.021708  Topic15  -5.4979   3.1959
430  sophisticated     51.019779     52.403398  Topic15  -5.5292   3.1951

[480 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
387       6  0.999160   ability
172       9  0.995829      able
277      11  0.994756     about
149      13  0.991184  academic
484      12  0.995024   achieve
...     ...       ...       ...
155       1  0.992658      year
221      10  0.999237     years
332       3  0.999684       you
514       8  0.992063    you_ll
18       10  0.998808      your

[435 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 8, 14, 15, 11, 13, 4, 9, 12, 5, 1, 10, 7, 6, 3])

Number of topics: 20

In [21]:
topic_description = Topic(n=20, doc=corpus_description, weight=True)
topic_description.train()
topic_description.visualize()

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15    -0.255951 -0.355633       1        1  13.387489
11    -0.242000  0.318654       2        1  12.082212
4     -0.190412  0.030884       3        1   8.859273
5     -0.177518  0.023700       4        1   8.341690
18    -0.089838  0.005404       5        1   5.257924
1     -0.057707  0.002785       6        1   4.754251
17    -0.042872  0.001889       7        1   4.384161
7     -0.026056  0.000988       8        1   4.314203
13     0.016637 -0.000771       9        1   3.817773
19     0.025580 -0.001041      10        1   3.694622
16     0.042747 -0.001569      11        1   3.670840
2      0.058853 -0.001958      12        1   3.446400
8      0.073649 -0.002282      13        1   3.400065
6      0.100540 -0.002779      14        1   3.153757
3      0.069610 -0.002188      15        1   3.124054
0      0.111049 -0.002930      16        1   3.069505
12     0.133499 -0.003192      17        1   2.962561
10     0.141611 -0.003269      18        1   2.951071
14     0.145608 -0.003300      19        1   2.749459
9      0.162971 -0.003393      20        1   2.578689, topic_info=      Term         Freq         Total Category  logprob  loglift
16     and  30681.00000  30681.000000  Default  30.0000  30.0000
44    data  15609.00000  15609.000000  Default  29.0000  29.0000
10      to  16924.00000  16924.000000  Default  28.0000  28.0000
39      in  11914.00000  11914.000000  Default  27.0000  27.0000
14     the  14385.00000  14385.000000  Default  26.0000  26.0000
..     ...          ...           ...      ...      ...      ...
12  skills      0.21689   1523.788606  Topic20 -10.5537  -5.1994
13      on      0.21689   3803.768818  Topic20 -10.5537  -6.1142
14     the      0.21689  14385.774882  Topic20 -10.5537  -7.4445
15     job      0.21689      2.432938  Topic20 -10.5537   1.2404
16     and      0.21689  30681.941354  Topic20 -10.5537  -8.2019

[935 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
387      11  0.998813  ability
277      12  0.996502    about
484      19  0.995278  achieve
338       9  0.997174   across
141      10  0.974838  actions
...     ...       ...      ...
234       9  0.998946  working
40       18  0.998301    world
155       2  0.991174     year
332       7  0.999511      you
18        6  0.998895     your

[237 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 12, 5, 6, 19, 2, 18, 8, 14, 20, 17, 3, 9, 7, 4, 1, 13, 11, 15, 10])

Number of topics: 30

In [23]:
topic_description = Topic(n=30, doc=corpus_description, weight=True)
topic_description.train()
topic_description.visualize()

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=         x    y  topics  cluster      Freq
topic                                     
0      0.0  0.0       1        1  3.333333
1      0.0  0.0       2        1  3.333333
28     0.0  0.0       3        1  3.333333
27     0.0  0.0       4        1  3.333333
26     0.0  0.0       5        1  3.333333
25     0.0  0.0       6        1  3.333333
24     0.0  0.0       7        1  3.333333
23     0.0  0.0       8        1  3.333333
22     0.0  0.0       9        1  3.333333
21     0.0  0.0      10        1  3.333333
20     0.0  0.0      11        1  3.333333
19     0.0  0.0      12        1  3.333333
18     0.0  0.0      13        1  3.333333
17     0.0  0.0      14        1  3.333333
16     0.0  0.0      15        1  3.333333
15     0.0  0.0      16        1  3.333333
14     0.0  0.0      17        1  3.333333
13     0.0  0.0      18        1  3.333333
12     0.0  0.0      19        1  3.333333
11     0.0  0.0      20        1  3.333333
10     0.0  0.0      21        1  3.333333
9      0.0  0.0      22        1  3.333333
8      0.0  0.0      23        1  3.333333
7      0.0  0.0      24        1  3.333333
6      0.0  0.0      25        1  3.333333
5      0.0  0.0      26        1  3.333333
4      0.0  0.0      27        1  3.333333
3      0.0  0.0      28        1  3.333333
2      0.0  0.0      29        1  3.333333
29     0.0  0.0      30        1  3.333333, topic_info=                 Term       Freq      Total Category  logprob  loglift
0          are_saying  26.000000  26.000000  Default  30.0000     30.0
7966           saving  26.000000  26.000000  Default  29.0000     29.0
7957        postingid  26.000000  26.000000  Default  28.0000     28.0
7958  experiencelevel  26.000000  26.000000  Default  27.0000     27.0
7959     locationname  26.000000  26.000000  Default  26.0000     26.0
...               ...        ...        ...      ...      ...      ...
25             offers   0.899274  26.978234  Topic30  -9.3882      0.0
26          extensive   0.899274  26.978234  Topic30  -9.3882      0.0
27           training   0.899274  26.978234  Topic30  -9.3882      0.0
28      opportunities   0.899274  26.978234  Topic30  -9.3882      0.0
29            through   0.899274  26.978234  Topic30  -9.3882      0.0

[930 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
11        1  0.037067  acquire
11        2  0.037067  acquire
11        3  0.037067  acquire
11        4  0.037067  acquire
11        5  0.037067  acquire
...     ...       ...      ...
18       26  0.037067     your
18       27  0.037067     your
18       28  0.037067     your
18       29  0.037067     your
18       30  0.037067     your

[1770 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 30])

**Conclusion:** Eine sehr interessante Beobachtung ist, wenn man die Anzahl an Themen auf 30 einstellt, dass dabei ein einziges Thema entsteht. Allgeimein, wenn man von 30 absteigend die Anzahl anschaut erkennt man, dass die Überlappung der Themen mit ansteigender Anzahl an Themen stärker wird und öfters bei mehreren Themen auftritt. In diesem Fall ist es generell sehr schwierig Struckur oder Merkmale aus diesen Themen zu finden, aber für diese Situation, würde man für die Anzahl 10 und für die 15 wählen, da alles drüber zu unübersichtlichkeit führt und als Personen keine Merkmale interpretieren oder beobachen kann.